In [1]:
import sys
import os
import pandas as pd
import re

In [2]:
sys.path.append("../")
from src.tokenize_data import create_tokenizer, tokenize_text, remove_stop_words

2023-05-05 16:36:44.110996: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-05 16:36:44.311052: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-05 16:36:44.312596: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 16:36:45.210608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package stopwords to /home/stefan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
DATA_FOLDER = "../data/genius_lyrics"
ARTISTS = ["Travis Scott", "Queen", "The Beatles"]

**getting the data into pandas**

In [4]:
df = pd.DataFrame()
for filename in os.listdir(DATA_FOLDER):
    artist_name = filename.split('.')[0]
    if artist_name in ARTISTS:
        temp_df = pd.read_csv(os.path.join(DATA_FOLDER, filename),
                usecols=['artist', 'title', 'lyrics'])
        df = pd.concat([df, temp_df])

df.head()

,artist,title,lyrics
0,Travis Scott,SICKO MODE,SICKO MODE Lyrics[Part I]\n\n[Intro: Drake]\nA...
1,Travis Scott,​goosebumps,​goosebumps Lyrics[Intro: Travis Scott]\nYeah\...
2,Travis Scott,BUTTERFLY EFFECT,BUTTERFLY EFFECT Lyrics[Intro]\nAll the commas...
3,Travis Scott,HIGHEST IN THE ROOM,HIGHEST IN THE ROOM Lyrics[Chorus]\nI got room...
4,Travis Scott,Antidote,Antidote Lyrics[Chorus]\nDon't you open up tha...


In [5]:
df.artist.value_counts()

artist
Travis Scott    100
The Beatles     100
Queen           100
Name: count, dtype: int64

In [6]:
sentence = "Mercy Lyrics[Intro: Fuzzy Jones]\nWell, it is"
sentence = re.sub("\[.*\]", "", sentence)  
sentence

'Mercy Lyrics\nWell, it is'

In [7]:
def clean_up_text(sentence):
    sentence = str(sentence).lower()
    sentence = re.sub("\[.*\]", "", sentence)  # remove sections markings in brakets
    sentence = re.sub(r'[^\w]', ' ', sentence)  # romove punctuation
    sentence = re.sub(r'[0-9]', '', sentence)  # remove numbers
    sentence = re.sub(r'\s[a-z]\s', ' ', sentence)  # remove single characters
    sentence = re.sub(r'^[a-z]\s', '', sentence)  # remove single characters from the start
    sentence = re.sub(r'\s+', ' ', sentence).strip()  # remove extra spaces

    sentence =  re.sub(r'.* lyrics', '', sentence)
    

    return sentence

df.lyrics = df.lyrics.apply(clean_up_text)
df.lyrics = df.lyrics.apply(remove_stop_words)

In [8]:
df = df.sort_values(by='title')
df = df.reset_index(drop=True)
df.head()

,artist,title,lyrics
0,Travis Scott,16 Chapels,know feel like floating night slow move groove...
1,Travis Scott,3500,lyrics bandana wrapped wrapped round head band...
2,Travis Scott,5% TINT,creeping window fore come outside got took end...
3,Travis Scott,90210,lyrics dean poppin yo travis mhm ooh ooh ooh o...
4,The Beatles,A Day in the Life,read news today oh boy lucky man made grade th...


**lets get those embeddings**

In [9]:
import torch
from transformers import RobertaTokenizer, RobertaModel

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')


def embed(sentence):
    tokens = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=512, return_token_type_ids=True, padding="max_length", truncation=True)
    input_ids = torch.tensor(tokens['input_ids']).unsqueeze(0)
    attention_mask = torch.tensor(tokens['attention_mask']).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    sentence_embedding = torch.mean(outputs[0], dim=1).squeeze()
    return sentence_embedding


df_dataset = pd.DataFrame()
df_dataset['vectors'] = df['lyrics'].apply(embed)
df_dataset.head()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,vectors
0,"[tensor(0.0428), tensor(-0.0517), tensor(0.122..."
1,"[tensor(-0.0514), tensor(-0.0310), tensor(0.10..."
2,"[tensor(-0.0152), tensor(-0.0518), tensor(0.13..."
3,"[tensor(0.0536), tensor(-0.0091), tensor(0.084..."
4,"[tensor(0.0712), tensor(-0.1449), tensor(0.043..."


In [10]:
df.artist.unique()

array(['Travis Scott', 'The Beatles', 'Queen'], dtype=object)

In [11]:
stt = {
    'Travis Scott': torch.tensor([1, 0, 0]),
    'The Beatles': torch.tensor([0, 1, 0]),
    'Queen': torch.tensor([0, 0, 1])
}

tts = {
    str(torch.tensor([1, 0, 0])): 'Travis Scott',
    str(torch.tensor([1, 0, 0])): 'The Beatles',
    str(torch.tensor([1, 0, 0])): 'Queen'
}

def encode_label(label: str):
    return stt[label]

def decode_label(label: torch.tensor) -> str:
    return tts[str(label)]

encode_label("Queen")

tensor([0, 0, 1])

In [12]:
df_dataset['artist'] = df['artist'].apply(encode_label)
df_dataset.head()

,vectors,artist
0,"[tensor(0.0428), tensor(-0.0517), tensor(0.122...","[tensor(1), tensor(0), tensor(0)]"
1,"[tensor(-0.0514), tensor(-0.0310), tensor(0.10...","[tensor(1), tensor(0), tensor(0)]"
2,"[tensor(-0.0152), tensor(-0.0518), tensor(0.13...","[tensor(1), tensor(0), tensor(0)]"
3,"[tensor(0.0536), tensor(-0.0091), tensor(0.084...","[tensor(1), tensor(0), tensor(0)]"
4,"[tensor(0.0712), tensor(-0.1449), tensor(0.043...","[tensor(0), tensor(1), tensor(0)]"


**model??**

let's see if you really trap

In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [78]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [79]:
class Net(nn.Module):

    def __init__(self, input_units, output_units):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_units, 1024)
        self.fc2 = nn.Linear(1024, 2048)
        self.fc3 = nn.Linear(2048, 4096)
        self.fc4 = nn.Linear(4096, 2048)
        self.fc5 = nn.Linear(2048, 2048)
        self.fc6 = nn.Linear(2048, 84)
        self.fc7 = nn.Linear(84, output_units)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.softmax(self.fc7(x), dim=1)
        return x

**turn of the autotune(copilot)\
lets see if you really rap**

In [80]:
# CONSTANTS
torch.manual_seed(42)
train_size = 0.8
learning_rate = 0.00001
num_epochs = 10

In [81]:
class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.stack(list(self.data[idx, 0])), torch.stack(list(self.data[idx, -1]))
    
split = int(train_size * len(df_dataset))
train_dataset = MyDataset(df_dataset[:split])
test_dataset = MyDataset(df_dataset[split:])

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [82]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

**training**

In [84]:
def multi_class_accuracy(preds: torch.Tensor, target: torch.Tensor) -> float:
    return ((torch.argmax(preds, dim=1) == torch.argmax(target, dim=1)) * 1).sum()\
        / target.shape[0]

In [85]:
def train_pass(
    model: torch.nn.Module,
    dataset: list,
    loss_function: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    accuracy_function,
    device: torch.device=device,
    display: bool=False) -> dict:
    
    model.train()
    train_acc, train_loss = 0, 0
    N = len(dataset)

    for batch, (X, y) in enumerate(dataset):
        X, y = X.to(device).float(), y.to(device).float()
        # forward pass
        y_pred = model(X)
        loss = loss_function(input=y_pred, target=y)

        train_acc += accuracy_function(preds=y_pred, target=y)
        train_loss += loss

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= N
    train_acc /= N

    if display:
        print(f'🔨Train loss: {train_loss:.4f} || Train acc: {train_acc:.4f}')

    results = {
        'train_loss': train_loss,
        'train_accuracy': train_acc
    }

    return results


def test_pass(
    dataset,
    model: torch.nn.Module,
    loss_function: torch.nn.Module,
    accuracy_function,
    device: torch.device=device,
    display: bool=False) -> dict:

    test_loss, test_acc = 0, 0
    N = len(dataset)

    for batch, (X, y) in enumerate(dataset):
        # sending data to right device
        X, y = X.to(device).float(), y.to(device).float()

        # forward pass
        test_pred = model(X)
        test_loss += loss_function(input=test_pred, target=y)

        # print(y_pred.shape, y.shape)
        test_acc += accuracy_function(preds=test_pred, target=y)

    test_loss /= N
    test_acc /= N
    if display:
        print(f'🧐Test loss: {test_loss:.4f} || Test acc: {test_acc:.4f}')

    results = {
        'test_loss': test_loss.cpu().detach().float(),
        'test_accuracy': test_acc.cpu().detach().float()
    }

    return results


for n, epoch in enumerate(range(num_epochs)):
    print(f'Epoch {n+1}/{num_epochs}')
    train_results = train_pass(
        model=model,
        dataset=train_dataloader,
        loss_function=criterion,
        optimizer=optimizer,
        accuracy_function=multi_class_accuracy,
        device=device,
        display=True
    )

    test_results = test_pass(
        dataset=test_dataloader,
        model=model,
        loss_function=criterion,
        accuracy_function=multi_class_accuracy,
        device=device,
        display=True
    )

Epoch 1/10
🔨Train loss: 0.5902 || Train acc: 0.9609
🧐Test loss: 0.7095 || Test acc: 0.8504
Epoch 2/10
🔨Train loss: 0.5875 || Train acc: 0.9648
🧐Test loss: 0.7203 || Test acc: 0.8103
Epoch 3/10
🔨Train loss: 0.5912 || Train acc: 0.9609
🧐Test loss: 0.7038 || Test acc: 0.8504
Epoch 4/10
🔨Train loss: 0.5904 || Train acc: 0.9648
🧐Test loss: 0.7106 || Test acc: 0.8170
Epoch 5/10
🔨Train loss: 0.5889 || Train acc: 0.9648
🧐Test loss: 0.7119 || Test acc: 0.8460
Epoch 6/10
🔨Train loss: 0.5842 || Train acc: 0.9688
🧐Test loss: 0.7148 || Test acc: 0.8170
Epoch 7/10
🔨Train loss: 0.5877 || Train acc: 0.9648
🧐Test loss: 0.7098 || Test acc: 0.8638
Epoch 8/10
🔨Train loss: 0.5876 || Train acc: 0.9648
🧐Test loss: 0.7096 || Test acc: 0.8170
Epoch 9/10
🔨Train loss: 0.5836 || Train acc: 0.9688
🧐Test loss: 0.7042 || Test acc: 0.8705
Epoch 10/10
🔨Train loss: 0.5836 || Train acc: 0.9688
🧐Test loss: 0.7084 || Test acc: 0.8214
